In [1]:
import pandas as pd

In [18]:
year = 2021
election = "first_round"

def create_polling_id(data, columns=[]):
    output = data[columns[0]].copy()

    for column in columns[1:]:
        output += "-" + data[column].astype(str)
    return output

In [19]:
df = pd.read_excel(
    f"../data/Peru/EG{year}_Resultados_Presidencial_{election}.xlsx",
    sheet_name="Nivel_Distrital"
)
df["polling_id"] = create_polling_id(df, ["Region", "Provincia", "Distrito"])
df.head()

,Region,Provincia,Distrito,Electores,Participación,% Participación,Ausentismo,% Ausentismo,Votos emitidos,% Votos emitidos,Votos válidos,% Votos válidos,Organización Política,Tipo Organización Política,Votos,% Votos,polling_id
0,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,20819,13619,0.6542,7200,0.3458,13619,0.6542,10948,0.5259,ACCION POPULAR,PARTIDO POLÍTICO,1338.0,0.1222,AMAZONAS-CHACHAPOYAS-CHACHAPOYAS
1,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,20819,13619,0.6542,7200,0.3458,13619,0.6542,10948,0.5259,ALIANZA PARA EL PROGRESO,PARTIDO POLÍTICO,1009.0,0.0922,AMAZONAS-CHACHAPOYAS-CHACHAPOYAS
2,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,20819,13619,0.6542,7200,0.3458,13619,0.6542,10948,0.5259,AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL,PARTIDO POLÍTICO,1132.0,0.1034,AMAZONAS-CHACHAPOYAS-CHACHAPOYAS
3,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,20819,13619,0.6542,7200,0.3458,13619,0.6542,10948,0.5259,DEMOCRACIA DIRECTA,PARTIDO POLÍTICO,36.0,0.0033,AMAZONAS-CHACHAPOYAS-CHACHAPOYAS
4,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,20819,13619,0.6542,7200,0.3458,13619,0.6542,10948,0.5259,EL FRENTE AMPLIO POR JUSTICIA. VIDA Y LIBERTAD,PARTIDO POLÍTICO,112.0,0.0102,AMAZONAS-CHACHAPOYAS-CHACHAPOYAS


In [20]:
df_candidates = pd.read_excel(
    f"../data/Peru/EG{year}_Candidatos_Presidencial_{election}.xlsx"
)
df_candidates["candidate"] = df_candidates.apply(lambda x: f"{x['Prenombres']} {x['Primer apellido']} {x['Segundo apellido']}", axis=1)
df_candidates = df_candidates[df_candidates["Cargo"] == "PRESIDENTE DE LA REPÚBLICA"]
df_candidates = df_candidates[["candidate", "Organización Política"]]
df_candidates.head()

,candidate,Organización Política
0,YONHY LESCANO ANCIETA,ACCION POPULAR
3,CESAR ACUÑA PERALTA,ALIANZA PARA EL PROGRESO
6,HERNANDO DE SOTO POLAR,AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL
9,ANDRES AVELINO ALCANTARA PAREDES,DEMOCRACIA DIRECTA
12,MARCO ANTONIO ARANA ZEGARRA,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD"


In [21]:
df = pd.merge(df, df_candidates, on="Organización Política")

In [22]:
df.head()

,Region,Provincia,Distrito,Electores,Participación,% Participación,Ausentismo,% Ausentismo,Votos emitidos,% Votos emitidos,Votos válidos,% Votos válidos,Organización Política,Tipo Organización Política,Votos,% Votos,polling_id,candidate
0,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,20819,13619,0.6542,7200,0.3458,13619,0.6542,10948,0.5259,ACCION POPULAR,PARTIDO POLÍTICO,1338.0,0.1222,AMAZONAS-CHACHAPOYAS-CHACHAPOYAS,YONHY LESCANO ANCIETA
1,AMAZONAS,CHACHAPOYAS,ASUNCION,285,114,0.4000,171,0.6000,114,0.4000,90,0.3158,ACCION POPULAR,PARTIDO POLÍTICO,17.0,0.1889,AMAZONAS-CHACHAPOYAS-ASUNCION,YONHY LESCANO ANCIETA
2,AMAZONAS,CHACHAPOYAS,BALSAS,1012,504,0.4980,508,0.5020,504,0.4980,356,0.3518,ACCION POPULAR,PARTIDO POLÍTICO,52.0,0.1461,AMAZONAS-CHACHAPOYAS-BALSAS,YONHY LESCANO ANCIETA
3,AMAZONAS,CHACHAPOYAS,CHETO,587,411,0.7002,176,0.2998,411,0.7002,296,0.5043,ACCION POPULAR,PARTIDO POLÍTICO,47.0,0.1588,AMAZONAS-CHACHAPOYAS-CHETO,YONHY LESCANO ANCIETA
4,AMAZONAS,CHACHAPOYAS,CHILIQUIN,732,324,0.4426,408,0.5574,324,0.4426,220,0.3005,ACCION POPULAR,PARTIDO POLÍTICO,28.0,0.1273,AMAZONAS-CHACHAPOYAS-CHILIQUIN,YONHY LESCANO ANCIETA


In [24]:
tt = df.groupby(["polling_id", "candidate"]).agg({"Votos": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate", "Votos"]]

tt["rank"] = tt.groupby(["polling_id"])["Votos"].rank("min", ascending=False).astype(int)

In [27]:
df_filtered = tt[["polling_id", "candidate", "Votos", "rank", "rate"]].copy()
df_filtered = df_filtered.rename(columns={"Votos": "value"})
df_filtered["candidate"] = df_filtered["candidate"].apply(lambda x: x.strip())

df_filtered.to_csv(f"../data_output/Peru/{year}_{election}.csv.gz", compression="gzip", index=False)

In [28]:
df.head()

,Region,Provincia,Distrito,Electores,Participación,% Participación,Ausentismo,% Ausentismo,Votos emitidos,% Votos emitidos,...,% Votos válidos,Organización Política,Tipo Organización Política,Votos_x,% Votos,polling_id,candidate,rate,Votos_y,rank
0,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,20819,13619,0.6542,7200,0.3458,13619,0.6542,...,0.5259,ACCION POPULAR,PARTIDO POLÍTICO,1338.0,0.1222,AMAZONAS-CHACHAPOYAS-CHACHAPOYAS,YONHY LESCANO ANCIETA,0.131099,1338.0,2
1,AMAZONAS,CHACHAPOYAS,ASUNCION,285,114,0.4000,171,0.6000,114,0.4000,...,0.3158,ACCION POPULAR,PARTIDO POLÍTICO,17.0,0.1889,AMAZONAS-CHACHAPOYAS-ASUNCION,YONHY LESCANO ANCIETA,0.207317,17.0,1
2,AMAZONAS,CHACHAPOYAS,BALSAS,1012,504,0.4980,508,0.5020,504,0.4980,...,0.3518,ACCION POPULAR,PARTIDO POLÍTICO,52.0,0.1461,AMAZONAS-CHACHAPOYAS-BALSAS,YONHY LESCANO ANCIETA,0.158537,52.0,2
3,AMAZONAS,CHACHAPOYAS,CHETO,587,411,0.7002,176,0.2998,411,0.7002,...,0.5043,ACCION POPULAR,PARTIDO POLÍTICO,47.0,0.1588,AMAZONAS-CHACHAPOYAS-CHETO,YONHY LESCANO ANCIETA,0.174721,47.0,1
4,AMAZONAS,CHACHAPOYAS,CHILIQUIN,732,324,0.4426,408,0.5574,324,0.4426,...,0.3005,ACCION POPULAR,PARTIDO POLÍTICO,28.0,0.1273,AMAZONAS-CHACHAPOYAS-CHILIQUIN,YONHY LESCANO ANCIETA,0.129630,28.0,3


In [29]:
df_location = df[["polling_id", "Region", "Provincia", "Distrito", "Electores", "Participación"]].drop_duplicates()
df_location = df_location.rename(columns={
    "Region": "region",
    "Provincia": "province",
    "Distrito": "district",
    "Electores": "voters",
    "Participación": "participation"
})

df_location.to_csv(f"../data_output/Peru/{year}_{election}_location.csv.gz", compression="gzip", index=False)